In [82]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
#from census import Census
#from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64


In [83]:
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=570719947456-53haph7mjc317oel1ujh0kmjbfd499jf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [115]:
from pydrive.auth import GoogleAuth

codebk = drive.CreateFile({'id': '1Lf7kV9_R-caFZV6_l8hMxAVjFSVFhvYl'})
codebk.GetContentFile('codebook.xlsx')  

codebk= pd.read_excel('codebook.xlsx', sheet_name = "predictors")
codebk.head()

/Users/bayerd01/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Variable,Description,Source,Notes,Include_BBL,Include_Box
0,geofips,census tract geo identifier,Census ACS,NaN,0.0,0.0
1,geofips_recode,"census tract geo identifier, changes county to...",Census ACS,NaN,0.0,0.0
2,totpop,total population,Census ACS,NaN,1.0,1.0
3,popdens,total population per square mile,Census ACS,NaN,1.0,1.0
4,areasqmile,area,Census ACS,NaN,1.0,1.0


In [116]:
predictors_full = codebk.loc[codebk["Include_BBL"] == 1, "Variable"].apply(lambda x: x.strip())
predictors_full

2             totpop
3            popdens
4         areasqmile
5              males
6            females
           ...      
102        assesstot
103        exempttot
104        yearbuilt
105      landuse_cat
106    bldgclass_cat
Name: Variable, Length: 90, dtype: object

In [85]:
# files in clean data google drive
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1jxSI_nq32-W9kPHKvQkzuNGJ5aIBD2j2' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

Title: analysis_bbl.csv, ID: 1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G
Title: analysis_bbl_noacs.csv, ID: 1h-tDpBSslfzqzsJe1ny_ujDnr68RKry3
Title: analysis_box.csv, ID: 1Vzjvta98IG9Hg4SfAGcd8JD8FaF3IXwy
Title: borobox_convex_hull.geojson, ID: 1SzBNj3Ba-v9MEwuCkZT9e9iAwVYUqx8r


## Loading Building Level Analysis File

In [86]:
df_bbl = drive.CreateFile({'id': '1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G'})
df_bbl.GetContentFile('analysis_bbl.csv')  

df_bbl= pd.read_csv('analysis_bbl.csv').drop(["Unnamed: 0"], axis = 1)
df_bbl.head()

,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,...,bldgclass_A0,bldgclass_A6,bldgclass_A8,bldgclass_CM,bbl,boroct2010,vacate_ind,borough,landuse,bldgclass
0,2013.0,5555.0,1.0,4.0,6.0,6.0,345150.0,1317150.0,0.0,1899.0,...,0.0,0.0,0.0,0.0,1012090027,1018100,0.0,MN,2.0,C2
1,2517.0,6895.0,1.0,4.0,10.0,10.0,427500.0,1244700.0,0.0,1891.0,...,0.0,0.0,0.0,0.0,1012090028,1018100,0.0,MN,2.0,C5
2,10570.0,127289.0,1.0,17.0,69.0,69.0,2616300.0,12195900.0,136178.0,1930.0,...,0.0,0.0,0.0,0.0,1012090037,1018100,0.0,MN,3.0,D4
3,3926.0,20358.0,1.0,6.0,25.0,25.0,490500.0,1929600.0,92325.0,1961.0,...,0.0,0.0,0.0,0.0,1012090049,1018100,0.0,MN,3.0,D4
4,5035.0,21258.0,1.0,6.0,12.0,12.0,585000.0,1414350.0,15640.0,1905.0,...,0.0,0.0,0.0,0.0,1012090051,1018100,0.0,MN,3.0,D4


In [87]:
df_bbl= pd.get_dummies(df_bbl, columns=["borough"])
borocols= ["borough_" + x for x in ["BK", "BX", "MN", "QN", "SI"]]

In [88]:
df_bbl

,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,...,bbl,boroct2010,vacate_ind,landuse,bldgclass,borough_BK,borough_BX,borough_MN,borough_QN,borough_SI
0,2013.0,5555.0,1.0,4.0,6.0,6.0,345150.0,1317150.0,0.0,1899.0,...,1012090027,1018100,0.0,2.0,C2,0,0,1,0,0
1,2517.0,6895.0,1.0,4.0,10.0,10.0,427500.0,1244700.0,0.0,1891.0,...,1012090028,1018100,0.0,2.0,C5,0,0,1,0,0
2,10570.0,127289.0,1.0,17.0,69.0,69.0,2616300.0,12195900.0,136178.0,1930.0,...,1012090037,1018100,0.0,3.0,D4,0,0,1,0,0
3,3926.0,20358.0,1.0,6.0,25.0,25.0,490500.0,1929600.0,92325.0,1961.0,...,1012090049,1018100,0.0,3.0,D4,0,0,1,0,0
4,5035.0,21258.0,1.0,6.0,12.0,12.0,585000.0,1414350.0,15640.0,1905.0,...,1012090051,1018100,0.0,3.0,D4,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765021,781000.0,1264820.0,13.0,17.0,1934.0,1947.0,29880450.0,72655200.0,72655200.0,1950.0,...,1001110100,1002500,0.0,4.0,D6,0,0,1,0,0
765022,23058.0,77910.0,1.0,6.0,36.0,36.0,832500.0,3132900.0,3132900.0,1920.0,...,1003250001,1001002,0.0,2.0,C4,0,0,1,0,0
765023,1206975.0,2948300.0,20.0,14.0,2391.0,2391.0,32398200.0,107938800.0,107938800.0,1962.0,...,1003230001,1001002,0.0,3.0,D3,0,0,1,0,0
765024,693945.0,1658649.0,18.0,14.0,1861.0,1861.0,26999100.0,95624100.0,95624100.0,1950.0,...,1003560001,1002000,0.0,3.0,D3,0,0,1,0,0


In [89]:
print(df_bbl.columns)

Index(['lotarea', 'bldgarea', 'numbldgs', 'numfloors', 'unitsres',
       'unitstotal', 'assessland', 'assesstot', 'exempttot', 'yearbuilt',
       ...
       'bbl', 'boroct2010', 'vacate_ind', 'landuse', 'bldgclass', 'borough_BK',
       'borough_BX', 'borough_MN', 'borough_QN', 'borough_SI'],
      dtype='object', length=164)


## Part I: Prediction (predictor variables only)

#### Define Predictors

In [98]:
# define predictors
df_bbl.columns
predictors = df_bbl[["totpop", "popdens", "areasqmile", "males", "females","pop18under","pop18_34","pop35_64","pop65plus",
              "pophispanic","popnhwhite","popnhblack","popnhaian","popnhasian","popnhhawainpi","popnhother"
              ,"popnhtwoplus","housing","housingoccupied","renterhousing","oc_halfless","oc_half_1","oc_1_onehalf"
              ,"oc_onehalf_2","oc_2plus","roc_halfless","roc_half_1","roc_1_onehalf","roc_onehalf_2","roc_2plus"
              ,"medianyearbuilt","mediangrossrent","medianhhincome","gini","pct18under","pct18_34","pct35_64"
              ,"pct65plus","pcthispanic","pctnhwhite","pctnhblack","pctnhaian","pctnhasian","pctnhhawainpi"
              ,"pctnhother","pctnhtwoplus","pctownerocch","pctrentocch","crowding","crowdingrenter","pctmale"
              ,"pcthgas","pcthelectricity","pcthoilker",'pcthcoal','pcthsolar','pcthotherf','pcthnofuel'
              ,'pctsamehouse','pctmvdsamecounty','pctmvdsdiffcounty','pctmvdsdiffstate','pctmvdabroad'
              ,'pop1yrplusinhh','pctpoprenterhh','AL','NB','tot_hpd_vio','hpd_fire_vio','hpd_b_c_vio'
              ,'viol_count_per_unit','hpd_fire_vio_per_unit','hpd_b_c_vio_per_unit','average_energy_usage'
              ,'average_energy_usage_mi'
              #,'holc_AB','holc_CD','holc_D' binary columns
              ,'lotarea','bldgarea','numbldgs','numfloors'
              ,'unitsres','unitstotal','assessland','assesstot','exempttot','yearbuilt'
                   #  ,'landuse_cat','bldgclass_cat' categorical
                    ]]



In [117]:
predictors

,totpop,popdens,areasqmile,males,females,pop18under,pop18_34,pop35_64,pop65plus,pophispanic,...,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt
0,8496.0,158948.70,0.053451,4072.0,4424.0,1037.0,2010.0,3202.0,2247.0,1400.0,...,2013.0,5555.0,1.0,4.0,6.0,6.0,345150.0,1317150.0,0.0,1899.0
1,8496.0,158948.70,0.053451,4072.0,4424.0,1037.0,2010.0,3202.0,2247.0,1400.0,...,2517.0,6895.0,1.0,4.0,10.0,10.0,427500.0,1244700.0,0.0,1891.0
2,8496.0,158948.70,0.053451,4072.0,4424.0,1037.0,2010.0,3202.0,2247.0,1400.0,...,10570.0,127289.0,1.0,17.0,69.0,69.0,2616300.0,12195900.0,136178.0,1930.0
3,8496.0,158948.70,0.053451,4072.0,4424.0,1037.0,2010.0,3202.0,2247.0,1400.0,...,3926.0,20358.0,1.0,6.0,25.0,25.0,490500.0,1929600.0,92325.0,1961.0
4,8496.0,158948.70,0.053451,4072.0,4424.0,1037.0,2010.0,3202.0,2247.0,1400.0,...,5035.0,21258.0,1.0,6.0,12.0,12.0,585000.0,1414350.0,15640.0,1905.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765021,5483.0,107977.70,0.050779,2335.0,3148.0,1075.0,1087.0,1803.0,1518.0,1699.0,...,781000.0,1264820.0,13.0,17.0,1934.0,1947.0,29880450.0,72655200.0,72655200.0,1950.0
765022,5157.0,66888.52,0.077098,2367.0,2790.0,944.0,925.0,2092.0,1196.0,3303.0,...,23058.0,77910.0,1.0,6.0,36.0,36.0,832500.0,3132900.0,3132900.0,1920.0
765023,5157.0,66888.52,0.077098,2367.0,2790.0,944.0,925.0,2092.0,1196.0,3303.0,...,1206975.0,2948300.0,20.0,14.0,2391.0,2391.0,32398200.0,107938800.0,107938800.0,1962.0
765024,4586.0,94058.32,0.048757,1675.0,2911.0,1090.0,926.0,1705.0,865.0,2561.0,...,693945.0,1658649.0,18.0,14.0,1861.0,1861.0,26999100.0,95624100.0,95624100.0,1950.0


#### Process Data

In [118]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize 

In [119]:
#standardize the data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(predictors)

In [102]:
scaled_features

array([[ 2.14046873e+00,  4.33506386e+00, -4.79120016e-01, ...,
         4.05926830e-01, -3.97483528e-02, -5.75460454e-01],
       [ 2.14046873e+00,  4.33506386e+00, -4.79120016e-01, ...,
         3.77512813e-01, -3.97483528e-02, -6.91870205e-01],
       [ 2.14046873e+00,  4.33506386e+00, -4.79120016e-01, ...,
         4.67244147e+00,  6.07152042e-02, -1.24372668e-01],
       ...,
       [ 4.83134496e-01,  1.01980154e+00, -4.02368386e-01, ...,
         4.22216530e+01,  7.95907043e+01,  3.41266336e-01],
       [ 1.99714954e-01,  1.99823776e+00, -4.94356053e-01, ...,
         3.73919760e+01,  7.05056938e+01,  1.66651710e-01],
       [-1.30920870e+00,  4.10252900e-01, -5.52173542e-01, ...,
         1.54616496e+01,  1.01541925e+00,  2.39407804e-01]])

#### Determine Number of Clusters

In [104]:
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture

In [112]:

range_n_clusters = range(2,5)


for n_clusters in range_n_clusters:
    clusterer = GaussianMixture(n_components=n_clusters, random_state=100)
    cluster_labels=clusterer.fit_predict(scaled_features)    
    silhouette_avg = silhouette_score(scaled_features, cluster_labels)
    print("For n_clusters = {},".format(n_clusters)+" the average silhouette_score is : {}".format(silhouette_avg))

For n_clusters = 2, the average silhouette_score is : 0.24961571808540126
For n_clusters = 3, the average silhouette_score is : 0.052963832745961274
For n_clusters = 4, the average silhouette_score is : 0.05576843088921439


#### Run Model

In [113]:
gmm = GaussianMixture(n_components=2,random_state=100)
res = gmm.fit(scaled_features)

print('Probabilities of belonging to cluster 1:\n',res.predict_proba(scaled_features)[:,1])
print('\nCluster centers:\n',res.means_)
print('\nCluster covariances:\n',res.covariances_)
print('\nLog-likelihood per sample:',res.score(scaled_features))

Probabilities of belonging to cluster 1:
 [4.48960843e-224 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
 0.00000000e+000 0.00000000e+000]

Cluster centers:
 [[ 0.56098119  0.95109895 -0.24731348  0.53972459  0.56601095  0.32221055
   0.83087004  0.43732426  0.27163427  0.461133    0.16632701  0.26903247
   0.04498144 -0.04262931  0.03452615 -0.03175074  0.28492843  0.7936162
  -0.22289315  1.12550501  0.6027454   0.82189694  0.43563428  0.62244565
   0.37297688  1.08821294  1.07452076  0.51366658  0.60817466  0.36190089
  -0.25547033  0.12706219 -0.05622135  0.63197245 -0.22257873  0.6575454
  -0.3346584  -0.28861318  0.25073044 -0.03834703  0.02984747 -0.0363177
  -0.24121846 -0.017931   -0.12660352  0.05609895 -0.84259635  0.84259635
   0.19036991  0.02921997 -0.06640355 -0.84842649  0.91404665  0.43960032
  -0.00900581 -0.05764563  0.60940035  0.59248591 -0.18307637  0.00715112
   0.08651361  0.37389528  0.15977072  0.53866023  0.75771665  0.28310381
   0.45342242  0.69909254

## Part II: Variable Relationships (including predictor variables)

In [114]:
df_bbl

,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,...,bbl,boroct2010,vacate_ind,landuse,bldgclass,borough_BK,borough_BX,borough_MN,borough_QN,borough_SI
0,2013.0,5555.0,1.0,4.0,6.0,6.0,345150.0,1317150.0,0.0,1899.0,...,1012090027,1018100,0.0,2.0,C2,0,0,1,0,0
1,2517.0,6895.0,1.0,4.0,10.0,10.0,427500.0,1244700.0,0.0,1891.0,...,1012090028,1018100,0.0,2.0,C5,0,0,1,0,0
2,10570.0,127289.0,1.0,17.0,69.0,69.0,2616300.0,12195900.0,136178.0,1930.0,...,1012090037,1018100,0.0,3.0,D4,0,0,1,0,0
3,3926.0,20358.0,1.0,6.0,25.0,25.0,490500.0,1929600.0,92325.0,1961.0,...,1012090049,1018100,0.0,3.0,D4,0,0,1,0,0
4,5035.0,21258.0,1.0,6.0,12.0,12.0,585000.0,1414350.0,15640.0,1905.0,...,1012090051,1018100,0.0,3.0,D4,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765021,781000.0,1264820.0,13.0,17.0,1934.0,1947.0,29880450.0,72655200.0,72655200.0,1950.0,...,1001110100,1002500,0.0,4.0,D6,0,0,1,0,0
765022,23058.0,77910.0,1.0,6.0,36.0,36.0,832500.0,3132900.0,3132900.0,1920.0,...,1003250001,1001002,0.0,2.0,C4,0,0,1,0,0
765023,1206975.0,2948300.0,20.0,14.0,2391.0,2391.0,32398200.0,107938800.0,107938800.0,1962.0,...,1003230001,1001002,0.0,3.0,D3,0,0,1,0,0
765024,693945.0,1658649.0,18.0,14.0,1861.0,1861.0,26999100.0,95624100.0,95624100.0,1950.0,...,1003560001,1002000,0.0,3.0,D3,0,0,1,0,0


In [121]:
# define variables for model
bbl_all = df_bbl[["totpop", "popdens", "areasqmile", "males", "females","pop18under","pop18_34","pop35_64","pop65plus",
              "pophispanic","popnhwhite","popnhblack","popnhaian","popnhasian","popnhhawainpi","popnhother"
              ,"popnhtwoplus","housing","housingoccupied","renterhousing","oc_halfless","oc_half_1","oc_1_onehalf"
              ,"oc_onehalf_2","oc_2plus","roc_halfless","roc_half_1","roc_1_onehalf","roc_onehalf_2","roc_2plus"
              ,"medianyearbuilt","mediangrossrent","medianhhincome","gini","pct18under","pct18_34","pct35_64"
              ,"pct65plus","pcthispanic","pctnhwhite","pctnhblack","pctnhaian","pctnhasian","pctnhhawainpi"
              ,"pctnhother","pctnhtwoplus","pctownerocch","pctrentocch","crowding","crowdingrenter","pctmale"
              ,"pcthgas","pcthelectricity","pcthoilker",'pcthcoal','pcthsolar','pcthotherf','pcthnofuel'
              ,'pctsamehouse','pctmvdsamecounty','pctmvdsdiffcounty','pctmvdsdiffstate','pctmvdabroad'
              ,'pop1yrplusinhh','pctpoprenterhh','AL','NB','tot_hpd_vio','hpd_fire_vio','hpd_b_c_vio'
              ,'viol_count_per_unit','hpd_fire_vio_per_unit','hpd_b_c_vio_per_unit','average_energy_usage'
              ,'average_energy_usage_mi'
              #,'holc_AB','holc_CD','holc_D' binary columns
              ,'lotarea','bldgarea','numbldgs','numfloors'
              ,'unitsres','unitstotal','assessland','assesstot','exempttot','yearbuilt'
                     ,'vacate_ind'
                   #  ,'landuse_cat','bldgclass_cat' categorical
                    ]]


In [122]:
bbl_all

,totpop,popdens,areasqmile,males,females,pop18under,pop18_34,pop35_64,pop65plus,pophispanic,...,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,vacate_ind
0,8496.0,158948.70,0.053451,4072.0,4424.0,1037.0,2010.0,3202.0,2247.0,1400.0,...,5555.0,1.0,4.0,6.0,6.0,345150.0,1317150.0,0.0,1899.0,0.0
1,8496.0,158948.70,0.053451,4072.0,4424.0,1037.0,2010.0,3202.0,2247.0,1400.0,...,6895.0,1.0,4.0,10.0,10.0,427500.0,1244700.0,0.0,1891.0,0.0
2,8496.0,158948.70,0.053451,4072.0,4424.0,1037.0,2010.0,3202.0,2247.0,1400.0,...,127289.0,1.0,17.0,69.0,69.0,2616300.0,12195900.0,136178.0,1930.0,0.0
3,8496.0,158948.70,0.053451,4072.0,4424.0,1037.0,2010.0,3202.0,2247.0,1400.0,...,20358.0,1.0,6.0,25.0,25.0,490500.0,1929600.0,92325.0,1961.0,0.0
4,8496.0,158948.70,0.053451,4072.0,4424.0,1037.0,2010.0,3202.0,2247.0,1400.0,...,21258.0,1.0,6.0,12.0,12.0,585000.0,1414350.0,15640.0,1905.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765021,5483.0,107977.70,0.050779,2335.0,3148.0,1075.0,1087.0,1803.0,1518.0,1699.0,...,1264820.0,13.0,17.0,1934.0,1947.0,29880450.0,72655200.0,72655200.0,1950.0,0.0
765022,5157.0,66888.52,0.077098,2367.0,2790.0,944.0,925.0,2092.0,1196.0,3303.0,...,77910.0,1.0,6.0,36.0,36.0,832500.0,3132900.0,3132900.0,1920.0,0.0
765023,5157.0,66888.52,0.077098,2367.0,2790.0,944.0,925.0,2092.0,1196.0,3303.0,...,2948300.0,20.0,14.0,2391.0,2391.0,32398200.0,107938800.0,107938800.0,1962.0,0.0
765024,4586.0,94058.32,0.048757,1675.0,2911.0,1090.0,926.0,1705.0,865.0,2561.0,...,1658649.0,18.0,14.0,1861.0,1861.0,26999100.0,95624100.0,95624100.0,1950.0,0.0


In [123]:
#standardize the data
scaler = StandardScaler()
scaled_all = scaler.fit_transform(bbl_all)

In [124]:
range_n_clusters = range(2,5)


for n_clusters in range_n_clusters:
    clusterer = GaussianMixture(n_components=n_clusters, random_state=100)
    cluster_labels=clusterer.fit_predict(scaled_all)    
    silhouette_avg = silhouette_score(scaled_all, cluster_labels)
    print("For n_clusters = {},".format(n_clusters)+" the average silhouette_score is : {}".format(silhouette_avg))

For n_clusters = 2, the average silhouette_score is : 0.2822837371643053
For n_clusters = 3, the average silhouette_score is : 0.05388875174095907
For n_clusters = 4, the average silhouette_score is : 0.05462284793306562


In [128]:
gmm = GaussianMixture(n_components=4,random_state=100)
res = gmm.fit(scaled_all)

print('Probabilities of belonging to cluster 1:\n',res.predict_proba(scaled_all)[:,1])
print('\nCluster centers:\n',res.means_)
print('\nCluster covariances:\n',res.covariances_)
print('\nLog-likelihood per sample:',res.score(scaled_all))

Probabilities of belonging to cluster 1:
 [0. 0. 0. ... 0. 0. 0.]

Cluster centers:
 [[ 2.36732305e-01  6.61007756e-01 -2.99713827e-01  2.47044203e-01
   2.21506870e-01  2.07520621e-01  4.64963360e-01  1.43639941e-01
  -7.17171278e-02  3.56983115e-01 -4.36649440e-02  1.54784813e-01
  -7.66978458e-02 -6.30271608e-02 -1.09504494e-01 -6.20377093e-02
   1.29139607e-01  3.31012937e-01 -3.30076808e-01  6.54489773e-01
   1.76782181e-01  3.87326612e-01  3.50988041e-01  4.03701636e-01
   3.12616944e-01  6.00565527e-01  6.25990261e-01  3.95383133e-01
   4.15648196e-01  3.02495769e-01 -2.98129608e-01  5.38511082e-02
  -1.85357367e-01  4.03378914e-01  4.04267739e-03  5.38962559e-01
  -3.05905216e-01 -4.10811509e-01  2.91921440e-01 -1.29646196e-01
   5.29877718e-02 -8.82247271e-02 -1.71995901e-01 -9.76883415e-02
  -8.48218590e-02  4.93388349e-02 -7.15009982e-01  7.15009982e-01
   3.26614732e-01  1.70384838e-01  3.60230107e-02 -4.40678276e-01
   4.73169567e-01  2.29187764e-01  1.20371640e-03 -7.6055